# ![s](https://s19.postimg.org/97wj4l35f/screenshot_1489112218.png)

# This tutorial is for SQLAlchemy 1.1.6

In [18]:
import sqlite3
from sqlalchemy import *
#The Python SQL Toolkit and Object Relational Mapper(ORM)
#http://docs.sqlalchemy.org/en/latest/intro.html#installation

In [9]:
db_engine = create_engine('sqlite:///sqlalchemy_example.db')

In [10]:
db_engine.echo=True 
#I want to see whats going in the background

In [20]:
metadata =MetaData(db_engine)

In [23]:
'''import pip
for i in pip.get_installed_distributions(local_only=True):
    print(i)'''

'import pip\nfor i in pip.get_installed_distributions(local_only=True):\n    print(i)'

In [24]:
user=Table('users',metadata,
          Column('ID',Integer,primary_key=True),
          Column('name', String(40)),
          Column('age', Integer)
          )

In [25]:
user.create()

2017-03-09 20:53:14,176 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2017-03-09 20:53:14,178 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2017-03-09 20:53:14,180 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:
CREATE TABLE users (
	"ID" INTEGER NOT NULL, 
	name VARCHAR(40), 
	age INTEGER, 
	PRIMARY KEY ("ID")
)




2017-03-09 20:53:14,191 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2017-03-09 20:53:14,500 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


### Intersting into User

In [28]:
user = Table('users', metadata, autoload=True)

In [26]:
i=user.insert()

In [29]:
i.execute(name='ABC', age=20)

2017-03-09 20:58:48,311 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, age) VALUES (?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, age) VALUES (?, ?)


2017-03-09 20:58:48,315 INFO sqlalchemy.engine.base.Engine ('ABC', 20)


INFO:sqlalchemy.engine.base.Engine:('ABC', 20)


2017-03-09 20:58:48,319 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [30]:
i.execute({'name': 'Superman', 'age': 30},
          {'name': 'Batman', 'age': 57},
          {'name': 'Acquaman', 'age': 38},
          {'name': 'Wonderwoman', 'age': 200},
          {'name': 'Flash', 'age': 25},
          {'name': 'Cyborg', 'age': 35}
         )

2017-03-09 21:00:45,361 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, age) VALUES (?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, age) VALUES (?, ?)


2017-03-09 21:00:45,362 INFO sqlalchemy.engine.base.Engine (('Superman', 30), ('Batman', 57), ('Acquaman', 38), ('Wonderwoman', 200), ('Flash', 25), ('Cyborg', 35))


INFO:sqlalchemy.engine.base.Engine:(('Superman', 30), ('Batman', 57), ('Acquaman', 38), ('Wonderwoman', 200), ('Flash', 25), ('Cyborg', 35))


2017-03-09 21:00:45,365 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [57]:
s=user.select()

In [58]:
conn=s.execute()

2017-03-09 21:10:48,216 INFO sqlalchemy.engine.base.Engine SELECT users."ID", users.name, users.age 
FROM users


INFO:sqlalchemy.engine.base.Engine:SELECT users."ID", users.name, users.age 
FROM users


2017-03-09 21:10:48,219 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


### Fetch one row, just like DB-API ``cursor.fetchone()``.

In [59]:
allrows=conn.fetchone()

In [60]:
print("ID: ", allrows[0])
print("Name: ", allrows.name)
print("Age: ", allrows['age'])

ID:  1
Name:  ABC
Age:  20


### Show all records in DB

In [61]:
for allrows in conn:
    print(allrows.name, 'is',allrows.age, 'years old')

Superman is 30 years old
Batman is 57 years old
Acquaman is 38 years old
Wonderwoman is 200 years old
Flash is 25 years old
Cyborg is 35 years old


## Echo Off

In [62]:
db_engine.echo=False

In [64]:
s=user.select()
conn=s.execute()
allrows=conn.fetchone()
for allrows in conn:
    print(allrows.name, 'is',allrows.age, 'years old')

Superman is 30 years old
Batman is 57 years old
Acquaman is 38 years old
Wonderwoman is 200 years old
Flash is 25 years old
Cyborg is 35 years old


In [65]:
def run(stmt):
    conn = stmt.execute()
    for rows in conn:
        print (rows)

## Finding names with 'man'

In [70]:
s=user.select(user.c.name.like('%man%'))# Most WHERE clauses can be constructed via normal comparisons
run(s)

(2, 'Superman', 30)
(3, 'Batman', 57)
(4, 'Acquaman', 38)
(5, 'Wonderwoman', 200)


## Finding age more than 50

In [71]:
s=user.select(user.c.age>50)
run(s)

(3, 'Batman', 57)
(5, 'Wonderwoman', 200)


## Python keywords like "and", "or", and "not" can't be overloaded, so
SQLAlchemy uses functions instead

In [79]:
s=user.select(and_(user.c.age>50,user.c.name.like('___ma%')))
run(s)

(3, 'Batman', 57)


### Or you could use &, | and ~ -- but watch out for priority!

In [82]:
s=user.select((user.c.age>50) & (user.c.name.like('___ma%')))
run(s)

(3, 'Batman', 57)


In [84]:
s = user.select(user.c.name.endswith('h'))
run(s)

(6, 'Flash', 25)
